In [1]:
import os
import time
import pandas as pd
from azure.storage.blob import BlobClient

In [2]:
# export the query results as csv files to map them into blob storage
files_to_get = pd.read_csv("/Users/senthil/Downloads/articles_new.csv")

files_to_get.head(5)

,ldxid,docxfilename,jsonresourcepath,revisedjsonresourcepath,document_path
0,28423,CMJ_20230052.docx,resources/28012024143006-27-_CMJ_20230052_v1.json,NaN,resources/20240128-142156.273320.docx.63842048...
1,28421,ETJ_230139.docx,resources/27012024035602-26-_ETJ_230139_v1.json,NaN,resources/20240127-034416.908960.docx.63841923...
2,28419,JME_230131.docx,resources/26012024144752-25-_JME_230131_v1.json,NaN,resources/20240126-144748.806370.docx.63841877...
3,28418,ECC_230516.docx,resources/26012024141851-25-_ECC_230516_v1.json,NaN,resources/20240126-141848.352610.docx.63841875...
4,28414,DIE_20230101.docx,resources/25012024155117-24-_DIE_20230101_v1.json,NaN,resources/20240125-154220.941960.docx.63841794...


In [6]:
# path to download the xml files
download_path = "/Users/senthil/Downloads/journals_data_element_prediction/dataset prep"
# define file to be downloaded
files_to_get["file_downloaded"] = files_to_get.docxfilename.apply(
    lambda x: os.path.exists(download_path + x))
# define new columns as per required
files_to_get["xml_file_path"] = ""
files_to_get["csv_file_path"] = ""
files_to_get["json_file_path"] = ""
files_to_get
# display total files after download
print ("Files to Get Shape", files_to_get.shape)
files_to_get

Files to Get Shape (1000, 9)


,ldxid,docxfilename,jsonresourcepath,revisedjsonresourcepath,document_path,file_downloaded,xml_file_path,csv_file_path,json_file_path
0,28423,CMJ_20230052.docx,resources/28012024143006-27-_CMJ_20230052_v1.json,NaN,resources/20240128-142156.273320.docx.63842048...,False,,,
1,28421,ETJ_230139.docx,resources/27012024035602-26-_ETJ_230139_v1.json,NaN,resources/20240127-034416.908960.docx.63841923...,False,,,
2,28419,JME_230131.docx,resources/26012024144752-25-_JME_230131_v1.json,NaN,resources/20240126-144748.806370.docx.63841877...,False,,,
3,28418,ECC_230516.docx,resources/26012024141851-25-_ECC_230516_v1.json,NaN,resources/20240126-141848.352610.docx.63841875...,False,,,
4,28414,DIE_20230101.docx,resources/25012024155117-24-_DIE_20230101_v1.json,NaN,resources/20240125-154220.941960.docx.63841794...,False,,,
...,...,...,...,...,...,...,...,...,...
995,22265,ELECT_20220234.docx,resources/28102023054646-300-_ELECT_20220234_v...,resources/28102023054646-300-_ELECT_20220234_r...,resources/20231028-054508.685560.docx.63834068...,False,,,
996,22264,CMJ_20220133.docx,resources/27102023102113-299-_CMJ_20220133_v1....,NaN,resources/20231027-093029.628480.docx.63833995...,False,,,
997,22263,BOHNS_20230015.docx,resources/27102023085426-299-_BOHNS_20230015_v...,resources/27102023085426-299-_BOHNS_20230015_r...,resources/20231027-085027.136870.docx.63833993...,False,,,
998,22256,JN_202322249.docx,resources/27102023053135-299-_JN_202322249_v1....,NaN,resources/20231027-053132.535960.docx.63833981...,False,,,


In [7]:
# check the file which are not found in blob storage
files_to_get["file_downloaded"] = files_to_get.docxfilename.apply(
    lambda x: os.path.exists(download_path + x))
files_to_get.index[files_to_get.file_downloaded == False]

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
      dtype='int64', length=1000)

In [8]:
# blob connection string
connection_string = "DefaultEndpointsProtocol=https;AccountName=vmlanstadtest01groupdiag;AccountKey=kqydQxk1AJn3cslG5sFZBQFdiJ5GUSmOSWNQsMCMf1lVuJNGA3Emy7W7+S+I7E8+s3AnWySD8heOim9rEpc7Pg==;EndpointSuffix=core.windows.net"

In [ ]:
# start downloading the files
# index to download
indexes_to_download = files_to_get.index[files_to_get.file_downloaded == False]
# iterate over the index to download
for i in indexes_to_download:
    if i % 50 == 0:
        print (i, "Sleeping")
        time.sleep(10)
    # define blob name
    blob_name = "epublishing/" + files_to_get.docx_path[i]
    # define download file path
    download_file = download_path + os.path.basename(files_to_get.docxfilename[i])
    print ("Downloading", i, blob_name, "To", download_file, sep = "--")
    # initialise BlobClient
    blob_client = BlobClient.from_connection_string(
    conn_str = connection_string,
    container_name = "blob-production-01", 
    blob_name = blob_name) 
    files_to_get["xml_file_path"][i] = download_file
    # start to download the file
    with open(download_file, "wb") as file:
        try:
            blob_client.download_blob().readinto(file)
        except:
            print ("Found On Error", i, blob_name, "To", download_file, sep = "--")

In [ ]:
import os
import urllib.parse
import re

# Define the base download path
download_base_path = download_path

# Start downloading the files
indexes_to_download = files_to_get.index[files_to_get.file_downloaded == False]

for i in indexes_to_download:
    if i % 50 == 0:
        print(i, "Sleeping")
        time.sleep(10)

    # Define blob names for different resources
    resource_names = {
        "docx": files_to_get.document_path[i],  # Corrected to .docx extension
        "json": files_to_get.jsonresourcepath[i],
        "revised_js": files_to_get.revisedjsonresourcepath[i]
    }

    # Remove .docx extension from docx_filename
    docx_filename_without_extension = os.path.splitext(files_to_get.docxfilename[i])[0]

    # Create the download folder based on docx_filename (without .docx extension)
    download_folder = os.path.join(download_base_path, docx_filename_without_extension)

    # Create the download folder if it doesn't exist
    os.makedirs(download_folder, exist_ok=True)

    for resource_type, blob_name in resource_names.items():
        if blob_name:
            # Convert blob_name to a string
            blob_name = str(blob_name)

            # Add the "epublishing/" prefix to the blob_name
            blob_name = "epublishing/" + blob_name

            # Define download file path based on resource_type
            if resource_type == "docx":
                resource_filename = f"{docx_filename_without_extension}.docx"  # Add .docx extension
            elif resource_type == "json":
                resource_filename = f"{docx_filename_without_extension}.json"
            elif resource_type == "revised_js":
                if pd.isna(files_to_get.revisedjsonresourcepath[i]):  # Skip if empty
                    continue
                resource_filename = f"{docx_filename_without_extension}_revised.json"
            else:
                # Handle other resource types if needed
                resource_filename = blob_name  # Use the blob name as the filename

            # Construct the full download path
            download_file = os.path.join(download_folder, resource_filename)

            # Print the final download path
            print("Downloading", i, blob_name, "To", download_file, sep="--")

            # Initialize BlobClient
            blob_client = BlobClient.from_connection_string(
                conn_str=connection_string,
                container_name="blob-production-01",
                blob_name=blob_name
            )

            # Start to download the file
            with open(download_file, "wb") as file:
                try:
                    blob_client.download_blob().readinto(file)
                except:
                    print("Found On Error", i, blob_name, "To", download_file, sep="--")
